In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns

from string import punctuation
from matplotlib import pyplot as plt

from sklearn import metrics, mixture, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, PCA
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import gensim
from gensim.models import word2vec

import nltk, string
from nltk.corpus import stopwords
from nltk.cluster import KMeansClusterer, cosine_distance, euclidean_distance
nltk.download('stopwords')
nltk.download('punkt')


# Bag of Words

There are many ways to transform text data to numeric vectors. In this task you will try to use two of them. One of the well-known approaches is a bag-of-words representation. To create this transformation, follow the steps:

1. Find N most popular words in train corpus and numerate them. Now we have a dictionary of the most popular words.  
2. For each title in the corpora create a zero vector with the dimension equals to N.  
3. For each text in the corpora iterate over words which are in the dictionary and increase by 1 the corresponding coordinate.

In [ ]:


def my_bag_of_words(text, words_to_index):
    """
    text: a string
    words_to_index: a list, train corpus words
    dict_size: size of the dictionary

    return a vector which is a bag-of-words representation of 'text'
    """

    dict_size = len(words_to_index)

    result_vector = np.zeros(dict_size)

    popular_words = enumerate(set(words_to_index))
    words_idx = {w:i for i,w in popular_words}

    for text in text.split():
        if text in words_idx:
            result_vector[words_idx[text]] += 1

    return result_vector

text = 'hi how are you'
words_to_index = ['hi', 'you', 'me', 'are']

my_bag_of_words(text, words_to_index)


# Word Vectorization

## Word2Vector

In [ ]:

def train_wordvec(docs, vector_size):
    # Tokenize docs into tokens and remove punctuations
    documents = []
    for doc in docs:
        tokens = [word.strip(string.punctuation) for word in doc.lower().split()]
        documents.append(tokens)

    # Train word vectors using gensim package
    model = gensim.models.Word2Vec(documents, vector_size=vector_size, window=5, min_count=1, workers=4)

    return model

def generate_doc_vector(docs, wv_model):


    # Tokenize each document into tokens
    tokenized_docs = []
    for doc in docs:
        tokens = [word.strip(string.punctuation) for word in doc.lower().split()]
        tokenized_docs.append(tokens)

    # Generate document vectors
    doc_vectors = []
    for doc in tokenized_docs:
        vectors = [wv_model.wv[token] for token in doc if token in wv_model.wv]
        if len(vectors) > 0:
            doc_vector = np.mean(vectors, axis=0)
        else:
            doc_vector = np.zeros(wv_model.vector_size)
        doc_vectors.append(doc_vector)

    vectors = np.array(doc_vectors)
    # Return document vectors as a numpy array
    return vectors


In [ ]:
# Here we will use our previous chatgpt dataset
data=pd.read_csv("./data/detect.csv")
x_train, x_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.2,random_state=0)

data.head()

documents=[
            [
                token.strip(punctuation).strip()
                 for token in nltk.word_tokenize(doc.lower())
                    if token not in punctuation and len(token.strip(punctuation).strip()) >= 2
            ]
            for doc in data["text"]
        ]

# use function
model = gensim.models.Word2Vec(documents, vector_size=300, window=5, min_count=5, workers=4)


In [ ]:

# print(model.wv['movie'], '\n')

# Top {n} word(s) with high relevance to {positive_vector} and low relevance to {negative_vector}: {top_n}
wv_correlation = lambda positive_vector=[], negative_vector=[], n=5: model.wv.most_similar(positive=positive_vector, negative=negative_vector, topn=n)
print(wv_correlation(['sound','music']), '\n')
print(wv_correlation(['sound','music'], ['film']), '\n')


# similarity between two tokens
wv_pos_sim = lambda token_a, token_b: model.wv.similarity(token_a, token_b)
print(wv_pos_sim('brass', 'acoustic'), '\n')
print(wv_pos_sim('movie','city'), '\n')

wv_outlier = lambda word_vector: model.wv.doesnt_match(word_vector)
print(wv_outlier(["sound", "music", "graphics", "actor", "book"]))



[('dance', 0.8091345429420471), ('pop', 0.8018935918807983), ('scene', 0.8004364371299744), ('recording', 0.7784169912338257), ('soul', 0.77668297290802)] 

[('mix', 0.6810025572776794), ('brass', 0.6776862144470215), ('acoustic', 0.6772572994232178), ('punk', 0.6729581356048584), ('influences', 0.664824903011322)] 

0.77276725 

-0.11576279 

actor


## Pretrained Word Vectors
- Google published pre-trained 300-dimensional vectors for 3 million words and phrases that were trained on Google News dataset (about 100 billion words)(https://code.google.com/archive/p/word2vec/)
- GloVe (Global Vectors for Word Representation): Pretained word vectors from different data sources provided by Standford https://nlp.stanford.edu/projects/glove/
- FastText by Facebook https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

In [ ]:

!pip install transformers
!pip install wget


In [ ]:
# from https://stackoverflow.com/questions/46433778/import-googlenews-vectors-negative300-bin

import wget
import gensim
from gensim.models import Word2Vec, KeyedVectors
from sklearn.decomposition import PCA
import torch
from transformers import BertTokenizer, BertModel, pipeline


In [ ]:

bin_url = "GoogleNews-vectors-negative300.bin"
gz_url = 'GoogleNews-vectors-negative300.bin.gz'
url = f'https://s3.amazonaws.com/dl4j-distribution/{gz_url}'

filename = wget.download(url)

f_in = gzip.open(gz_url, 'rb')
f_out = open(bin_url, 'wb')
f_out.writeline(f_in)

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [7]:

model.most_similar(positive=['women','king'], negative='man')


[('queen', 0.4827326238155365),
 ('queens', 0.466781347990036),
 ('kumaris', 0.4653734564781189),
 ('kings', 0.4558638632297516),
 ('womens', 0.422832190990448),
 ('princes', 0.4176960587501526),
 ('Al_Anqari', 0.41725507378578186),
 ('concubines', 0.4011078476905823),
 ('monarch', 0.3962482810020447),
 ('monarchy', 0.39430150389671326)]

In [8]:

# Load the BERT tokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens
print (tokenized_text)
# The original word has been split into smaller subwords and characters.
# The two hash signs preceding some of these subwords are just our tokenizer’s way to denote that this subword
# or character is part of a larger word and preceded by another subword.
# this way some contextual meaning of the original word will be retained.

# check out contents of BERT’s vocabulary
list(tokenizer.vocab.keys())[5000:5020]


Loading BERT tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]']


['knight',
 'lap',
 'survey',
 'ma',
 '##ow',
 'noise',
 'billy',
 '##ium',
 'shooting',
 'guide',
 'bedroom',
 'priest',
 'resistance',
 'motor',
 'homes',
 'sounded',
 'giant',
 '##mer',
 '150',
 'scenes']

In [9]:

unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Artificial Intelligence [MASK] take over the world.")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.3182409107685089,
  'token': 2064,
  'token_str': 'can',
  'sequence': 'artificial intelligence can take over the world.'},
 {'score': 0.18299664556980133,
  'token': 2097,
  'token_str': 'will',
  'sequence': 'artificial intelligence will take over the world.'},
 {'score': 0.05600148066878319,
  'token': 2000,
  'token_str': 'to',
  'sequence': 'artificial intelligence to take over the world.'},
 {'score': 0.04519490897655487,
  'token': 2015,
  'token_str': '##s',
  'sequence': 'artificial intelligences take over the world.'},
 {'score': 0.045153163373470306,
  'token': 2052,
  'token_str': 'would',
  'sequence': 'artificial intelligence would take over the world.'}]

In [12]:

import pandas as pd
import nltk, string

nltk.download('punkt')

# Load data
data=pd.read_csv('/data/amazon_reviews.csv')
data.columns=['label','text']
data

# tokenize each document into a list of unigrams
# strip punctuations and leading/trailing spaces from unigrams
# only unigrams with 2 or more characters are taken
sentences=[ [token.strip(string.punctuation).strip() 
             for token in nltk.word_tokenize(doc.lower()) 
                 if token not in string.punctuation and 
                 len(token.strip(string.punctuation).strip())>=2]
             for doc in data["text"]]

print(sentences[0:2])



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['this', 'is', 'little', 'longer', 'and', 'more', 'detailed', 'than', 'the', 'first', 'two', 'books', 'in', 'the', 'series', 'however', 'have', 'enjoyed', 'each', 'new', 'aspect', 'of', 'the', 'exciting', 'fantasy', 'universe'], ['only', 'michelle', 'branch', 'save', 'this', 'album', 'all', 'guys', 'play', 'along', 'with', 'unenthusiastic', 'beat', 'even', 'karl']]


In [13]:
# use our data
data=data.iloc[:100]
sentences=data["text"].values
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

input_ids = []
attention_masks = []
max_len =50

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                                       # Sentence to encode
                        add_special_tokens = True,                  # Add '[CLS]' and '[SEP]'
                        max_length = max_len,                       # Pad & truncate all sentences
                        pad_to_max_length = True,
                        return_attention_mask = True,               # Construct attn. masks
                        return_tensors = 'pt',                      # Return pytorch tensors
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])


# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 Original:  This is a little longer and more detailed than the first two books in the series. However, I have enjoyed each new aspect of the exciting fantasy universe.
Tokenized:  ['this', 'is', 'a', 'little', 'longer', 'and', 'more', 'detailed', 'than', 'the', 'first', 'two', 'books', 'in', 'the', 'series', '.', 'however', ',', 'i', 'have', 'enjoyed', 'each', 'new', 'aspect', 'of', 'the', 'exciting', 'fantasy', 'universe', '.']
Token IDs:  [2023, 2003, 1037, 2210, 2936, 1998, 2062, 6851, 2084, 1996, 2034, 2048, 2808, 1999, 1996, 2186, 1012, 2174, 1010, 1045, 2031, 5632, 2169, 2047, 7814, 1997, 1996, 10990, 5913, 5304, 1012]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [14]:


# Load pre-trained model (weights)
bert_model = BertModel.from_pretrained('bert-base-uncased',
                                    #output_attentions = False, # Whether the model returns attentions weights.
                                    output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

## Put the model in "evaluation" mode, meaning feed-forward operation.
bert_model.eval()
with torch.no_grad():

    outputs = bert_model(input_ids)
    #the third item will be the hidden states from all layers.
    hidden_states = outputs[2]

print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0
print ("Number of batches:", len(hidden_states[layer_i]))
#The second dimension, the batch size, is used when submitting multiple sentences to the model at once
batch_i = 0
print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0
print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0
print ("Number of batches:", len(hidden_states[layer_i]))
#The second dimension, the batch size, is used when submitting multiple sentences to the model at once
batch_i = 0
print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0
print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))



# get the last four layers
token_embeddings = torch.stack(hidden_states[-4:], dim=0)
print(token_embeddings.size())

# permute axis
token_embeddings = token_embeddings.permute(1,2,0,3)
print(token_embeddings.size())

# take the mean of the last 4 layers
token_embeddings = token_embeddings.mean(axis=2)
print(token_embeddings.size())



We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 100
Number of tokens: 50
Number of hidden units: 768
Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 100
Number of tokens: 50
Number of hidden units: 768
torch.Size([4, 100, 50, 768])
torch.Size([100, 50, 4, 768])
torch.Size([100, 50, 768])


In [15]:
token_embeddings[0,0,:]

tensor([-1.1674e-01, -5.8241e-01,  4.6175e-02, -2.3729e-01, -5.1516e-01,
        -1.8817e-01,  3.6290e-01,  2.7166e-01,  1.7761e-01, -8.4111e-01,
        -1.9935e-01,  3.3611e-01,  2.5841e-01,  5.8463e-01,  7.7076e-01,
         4.6129e-01, -1.8080e-01,  5.4306e-01,  3.8711e-01, -2.4097e-01,
         3.7831e-01,  4.6802e-01,  8.4769e-01,  1.0741e-01,  1.6989e-01,
        -2.8467e-01, -5.7680e-01, -1.5120e-01, -4.5195e-01,  7.4451e-02,
        -4.1504e-01,  6.1160e-02,  3.2209e-02,  2.4624e-01,  6.4040e-01,
        -2.9038e-01, -1.0277e-02, -2.5516e-01,  1.7088e-01,  3.1873e-02,
        -4.9831e-01, -2.2552e-01,  4.3180e-01, -2.0207e-01,  3.9851e-01,
        -3.9309e-01, -4.3959e+00,  1.9462e-01, -2.2815e-01,  2.0750e-01,
         2.5107e-01, -3.9301e-01,  6.9134e-02,  2.4547e-01,  5.4014e-01,
         8.6356e-01, -4.0756e-01, -4.7488e-01,  5.7281e-01,  1.6575e-04,
         3.8241e-01,  2.6872e-01, -3.5878e-01, -1.2718e-01, -2.0059e-01,
         1.6424e-01,  5.5433e-01, -1.9884e-01, -3.5